# hw4 나무 구조에 대한 함수형 프로그래밍 좀더 (제출기한 11/5 밤, 6점)

* 과목: 프로그래밍언어
* 이름: 
* 학번:

이후에 나오는 다음 세 함수의 정의를 완성하고,
완성한 각각의 함수가 해당 나무구조에 대해 올바른 동작을 하는 것을 보여주는 테스트 케이스를 최소한 하나씩 실행해 보라.
```haskell
preOrder :: Tree a -> [a]   -- 1점

lvOrd2 :: Tree2 a -> [[a]]  -- 2점

lvOrd :: Tree a -> [[a]]    -- 3점
```

In [1]:
-- 이진트리
data Tree2 a
  = Null2
  | Node2 a (Tree2 a) (Tree2 a)
  deriving Show

-- 임의 개수 가지치는 나무
data Tree a = Node a [Tree a]  deriving Show

In [2]:
-- Tree2의 중위순서(inorder)로 원소를 나열하는 리스트를 계산하자
inOrder2 :: Tree2 a -> [a]
inOrder2 Null2           = []
inOrder2 (Node2 x t1 t2) = inOrder2 t1 ++ [x] ++ inOrder2 t2

In [3]:
-- Tree2의 전위순서(preorder)로 원소를 나열하는 리스트를 계산하자
preOrder2 :: Tree2 a -> [a]
preOrder2 Null2           = []
preOrder2 (Node2 x t1 t2) = x : preOrder2 t1 ++ preOrder2 t2

In [4]:
tree2 = Node2 1
          (Node2 2 (Node2 4 Null2 Null2)
                   (Node2 5 Null2 Null2))
          (Node2 3 (Node2 6 Null2 Null2)
                   (Node2 7 Null2 Null2))

inOrder2 tree2

preOrder2 tree2

[4,2,5,1,6,3,7]

[1,2,4,5,3,6,7]

In [7]:
-- Null2가 아닌 Tree2를 Tree로 변환
fromTree2toTree Null2 = error "Null2 cannot be Tree"
fromTree2toTree (Node2 x t1 t2) = Node x ts
  where
  ts = case (t1, t2) of
         (Null2, Null2) -> []
         (Null2, _    ) -> [t2']
         (_,     Null2) -> [t1']
         (_,     _    ) -> [t1',t2']    
  t1' = fromTree2toTree t1
  t2' = fromTree2toTree t2

In [8]:
-- 위 함수를 case를 쓰는 대신에 좀더 깔끔하게 정리해
-- 똑같은 계산을 하는 함수를 다음과 같이 정의할 수 있다
fromTree2toTree Null2 = error "Null2 cannot be Tree"
fromTree2toTree (Node2 x Null2 Null2) = Node x []
fromTree2toTree (Node2 x Null2 t2) = Node x [fromTree2toTree t2]
fromTree2toTree (Node2 x t1 Null2) = Node x [fromTree2toTree t1]
fromTree2toTree (Node2 x t1 t2) = Node x [fromTree2toTree t1, fromTree2toTree t2]

In [9]:
tree = fromTree2toTree tree2

tree

Node 1 [Node 2 [Node 4 [],Node 5 []],Node 3 [Node 6 [],Node 7 []]]

# hw4 나무 구조에 대한 함수형 프로그래밍 좀더 (제출기한 11/5 밤, 6점)
이후에 나오는 다음 세 함수의 정의를 완성하고,
완성한 각각의 함수가 해당 나무구조에 대해 올바른 동작을 하는 것을 보여주는 테스트 케이스를 최소한 하나씩 실행해 보라.
```haskell
preOrder :: Tree a -> [a]   -- 1점

lvOrd2 :: Tree2 a -> [[a]]  -- 2점

lvOrd :: Tree a -> [[a]]    -- 3점
```

## 임의 개수로 가지치는 나무구조에 대한 전위 탐색

아래는 Tree의 전위(preorder) 탐색 순서대로 나열한 리스트를 계산하는 함수를 정의하려는 시도이다.
정답에 가깝지만 타입 오류가 난다. 타입 오류의 원인이 무엇인지 생각해 보고 프로그램을 올바르게 수정하라.

In [10]:
preOrder :: Tree a -> [a]
preOrder (Node x ts) = x : [preOrder t | t<-ts]

<interactive>:2:29: error:
    • Couldn't match expected type ‘a’ with actual type ‘[a]’
      ‘a’ is a rigid type variable bound by
        the type signature for:
          preOrder :: forall a. Tree a -> [a]
        at <interactive>:1:1-25
    • In the expression: preOrder t
      In the second argument of ‘(:)’, namely ‘[preOrder t | t <- ts]’
      In the expression: x : [preOrder t | t <- ts]
    • Relevant bindings include
        t :: Tree a (bound at <interactive>:2:42)
        ts :: [Tree a] (bound at <interactive>:2:18)
        x :: a (bound at <interactive>:2:16)
        preOrder :: Tree a -> [a] (bound at <interactive>:2:1)

## 이진트리에 대한 레벨 순서 탐색

이진트리의 원소를 level order로 나열한 리스트를 계산하는 `levelOrder2` 함수를 정의해 보자.
이 함수는 이진트리의 같은 레벨끼리 리스트로 구분지어주고 뿌리에 가까운 레벨부터 나열하는
리스트의 리스트를 계산하는 `lvOrd2` 함수를 이용하면 손쉽게 정의할 수 있다.
`levelOrder2`는 `lvOrd2`의 결과를 평평한 리스트로 이어붙이는 `concat` 함수와의 합성함수로 정의할 수 있다.

예컨대, `lvOrd2`와 `levelOrd` 함수의 실행예는 다음과 같다.
```haskell
lvOrd2 tree2 == [[1],[2,3],[4,5,6,7]]
levelOrder2 tree2 == [1,2,3,4,5,6,7]
```
참고로
```haskell
concat [[1],[2,3],[4,5,6,7]] == [1,2,3,4,5,6,7]
```
이제 `lvOrd2` 함수를 작성해 보라.

In [11]:
levelOrder2 :: Tree2 a -> [a]
levelOrder2 = concat . lvOrd2

lvOrd2 :: Tree2 a -> [[a]]
lvOrd2 Null2           = []
lvOrd2 (Node2 x t1 t2) = undefined

## 임의 개수로 가지치는 나무구조에 대한 레벨 순서 탐색

마찬가지로 임의의 개수로 가지치는 나무구조에 대해 마찬가지 계산을 하는 `levelOrder` 함수를 `lvOrd` 함수를 이용해 정의할 수 있다.

이제 `lvOrd` 함수를 작성해 보라.

In [12]:
levelOrder :: Tree a -> [a]
levelOrder = concat . lvOrd

lvOrd :: Tree a -> [[a]]
lvOrd (Node x ts) = undefined

----
위 과제를 하는 데 있어 도움이 될만한 힌트나 하스켈 라이브러리 함수 등에 대해서는 이후 수업시간을 활용하여 보충 설명할 예정이다.
힌트 없이 미리 도전해 보고자 하는 분들을 위해 중간고사 전에 미리 과제를 제출하였다.